In [7]:
# imports to open and store the json datasets into mongodb

import json
import gzip
from pymongo import MongoClient

In [1]:
# establishing connection to mongodb and database
client = "mongodb://localhost:27017"
database = "products"

In [35]:
# function to read and load the gzip json files into mongodb

def mongodb_insert(client, database, collection, input_file):
    # connect to MongoDB
    client = MongoClient(client)
    db = client[database]
    collection = db[collection]

    # doc counter
    count = 0

    # open the gzipped json file
    with gzip.open(input_file) as file:
        for line in file:
            try:
                # parse each line of JSON and insert into MongoDB
                data = json.loads(line.strip())
                collection.insert_one(data)
                count +=1
            except Exception as e:
                print(f"Error inserting data into MongoDB: {e}")

    # close the connection
    client.close()
    print("inserted", count, "docs")

In [97]:
# inserting product metadata

collection = "metadata"
input_file = "../project_data/meta_Home_and_Kitchen.json.gz"
# mongodb_insert(client, database, collection, input_file)

inserted 1300540 docs


In [36]:
# inserting product review data
collection = "reviews"
input_file = "../project_data/Home_and_Kitchen.json.gz"
mongodb_insert(client, database, collection, input_file)

inserted 21928568 docs


In [17]:
# function for performing a substring query on the database

def mongodb_substring_query(client, database, collection, substring, query, projection):
    client = MongoClient(client)
    db = client[database]
    collection = db[collection]
    result_list = []

    result = collection.find(query,projection)

    for item in result:
        result_list.append(item)
    
    client.close()
    return result_list

In [18]:
# querying the metadata for items in the category of desk chairs and retreiving asin (ID#)

collection = "metadata"
substring = "desk chairs"

query = {"category":{"$elemMatch":{"$regex":substring, "$options":"i"}}}
projection = {"_id":0, "asin":1}

asin_list = mongodb_substring_query(client, database, collection, substring, query, projection)

In [21]:
len(asin_list)

2088

In [22]:
# converting dict values to strings
asins = [item['asin'] for item in asin_list]

# storing the asin list in a text file
with open('../project_data/asin_list.txt', 'w') as file:
    for value in asins:
        file.write(value+'\n')

In [38]:
# reading in the stored asin values
asin_list = []
with open('asin_list.txt', 'r') as file:
    for line in file:
        asin_list.append(line.strip())

In [30]:
# function to create index on the asin field in the metadata collection

def mongodb_create_index(client, database, collection, index_field):
    client = MongoClient(client)
    db = client[database]
    collection = db[collection]

    collection.create_index((index_field))

    client.close()

In [31]:
# creating index on asin field in mongodb metadata collection

collection="metadata"
index_field = "asin"

mongodb_create_index(client, database, collection, index_field)

In [37]:
# creating index on asin field in mongodb metadata collection

collection="reviews"
index_field = "asin"

mongodb_create_index(client, database, collection, index_field)

In [40]:
# import pandas
import pandas as pd

In [44]:
# function for performing a substring query on the database

def mongodb_asin_query(client, database, collection, asin_list):
    client = MongoClient(client)
    db = client[database]
    collection = db[collection]

    output = []
    count = 0

    result = collection.find({"asin":{"$in": asin_list}})

    for item in result:
        output.append(item)
        count +=1
        
    client.close()
    return output
    print("retrieved", count, "docs")

In [48]:
# collecting desired products from products collection

collection = "metadata"
df = pd.DataFrame(mongodb_asin_query(client, database, collection, asin_list))

In [53]:
# save as csv
df = df.loc[:,['asin','title','description','brand','date','price']]
df.to_csv("../project_data/products.csv")

In [54]:
# collect corresponding reviews for products from reviews collection

collection = "reviews"
df = pd.DataFrame(mongodb_asin_query(client, database, collection, asin_list))

In [57]:
# save to csv
df = df.loc[:,['asin','reviewerID','overall','summary','reviewText']]
df.to_csv("../project_data/reviews.csv")